# Transfer Learning in NMT with attention

Telugu English Parallel corpus

In [7]:
en_te_train = open('./en-te/en-te/train.en').readlines()
te_train = open('./en-te/en-te/train.te', encoding='utf-8').readlines()

for pair in list(zip(te_train[:5], en_te_train[:5])):
    print(f'{pair[0]}\t{pair[1]}')

మళ్లీ ఉదయిస్తాడు.
	Rise again.

యెహోవా కృపను మనమెలా మహిమపరచవచ్చు?
	How do we glorify Jehovahs undeserved kindness?

ఆర్థికంగా కూడా భారత్‌ వే గంగా పయనిస్తున్నది.
	India also continues to push back economically.

‘విద్యార్థులను చూస్తుంటే నాకు చిన్నప్పటి రోజులు గుర్తుకొస్తున్నాయి.
	I remember my childhood days.

ఆర్థిక లావాదేవీలన్నీ ఆన్‌లైన్‌లోనే
	All transactions are made online.



Kannada English Parallel corpus

In [9]:
en_kn_train = open('./en-kn/en-kn/train.en').readlines()
ka_train = open('./en-kn/en-kn/train.kn', encoding='utf-8').readlines()

for pair in list(zip(te_train[:5], en_kn_train[:5])):
    print(f'{pair[0]}\t{pair[1]}')

మళ్లీ ఉదయిస్తాడు.
	Such videos are aplenty on social media channels.

యెహోవా కృపను మనమెలా మహిమపరచవచ్చు?
	This incident happened within the limits of Mudhol Police Station.

ఆర్థికంగా కూడా భారత్‌ వే గంగా పయనిస్తున్నది.
	The situation is currently under control, he said.

‘విద్యార్థులను చూస్తుంటే నాకు చిన్నప్పటి రోజులు గుర్తుకొస్తున్నాయి.
	Choose a Bookmark to be your Home Page. If you choose a folder, the Bookmarks in that folder will be opened in Tabs.

ఆర్థిక లావాదేవీలన్నీ ఆన్‌లైన్‌లోనే
	Several social organisations are also being involved.

